In [4]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [70]:
#計測地点のosm_idを指定
start_osm = 266396723
mid_osm = [232919754,651358177]
end_osm = 637167088
start_mesh = 50325662142
end_mesh = 50325652441

In [71]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [72]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun_2e_14"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [73]:
datedir = "230225"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値10.5460986905934距離0.14468426940260257 km
存在メッシュ：50325662142値6.45距離0.0 km
存在メッシュ：50325662213値11.691926243613176距離0.1851577137438622 km
存在メッシュ：50325662231値10.5460986905934距離0.14468426940260257 km
存在メッシュ：50325662142値6.45距離0.0 km
存在メッシュ：50325662232値14.642197381027792距離0.2893685387995886 km
存在メッシュ：50325662124値9.721018646598361距離0.11554041512042086 km
存在メッシュ：50325662213値11.691926243613176距離0.1851577137438622 km
存在メッシュ：50325662231値10.5460986905934距離0.14468426940260257 km
存在メッシュ：50325662232値14.642197381027792距離0.2893685387995886 km
存在メッシュ：50325662213値11.691926243613176距離0.1851577137438622 km
存在メッシュ：50325662142値6.45距離0.0 km
存在メッシュ：50325662232値14.642197381027792距離0.2893685387995886 km
存在メッシュ：50325662231値10.5460986905934距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値14.443564215676624距離0.14468426940260257 km
存在メッシュ：50325662142値9.1距離0.0 km
存在メッシュ：50325662213値15.9383531776

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.6006395286131距離0.14468426940260257 km
存在メッシュ：50325662142値30.383333333333333距離0.0 km
存在メッシュ：50325662213値52.41694531554025距離0.1851577137438622 km
存在メッシュ：50325662231値47.6006395286131距離0.14468426940260257 km
存在メッシュ：50325662142値30.383333333333333距離0.0 km
存在メッシュ：50325662232値64.81794572322451距離0.2893685387995886 km
存在メッシュ：50325662124値44.13254541611028距離0.11554041512042086 km
存在メッシュ：50325662213値52.41694531554025距離0.1851577137438622 km
存在メッシュ：50325662231値47.6006395286131距離0.14468426940260257 km
存在メッシュ：50325662232値64.81794572322451距離0.2893685387995886 km
存在メッシュ：50325662213値52.41694531554025距離0.1851577137438622 km
存在メッシュ：50325662142値30.383333333333333距離0.0 km
存在メッシュ：50325662232値64.81794572322451距離0.2893685387995886 km
存在メッシュ：50325662231値47.6006395286131距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値46.57090717741513距離0.14468426940260257 km
存在メッシュ：50325662142値28.65距離0.

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.98777611049852距離0.14468426940260257 km
存在メッシュ：50325662142値30.766666666666666距離0.0 km
存在メッシュ：50325662213値52.805145804160134距離0.1851577137438622 km
存在メッシュ：50325662231値47.98777611049852距離0.14468426940260257 km
存在メッシュ：50325662142値30.766666666666666距離0.0 km
存在メッシュ：50325662232値65.20888555366187距離0.2893685387995886 km
存在メッシュ：50325662124値44.518915906961354距離0.11554041512042086 km
存在メッシュ：50325662213値52.805145804160134距離0.1851577137438622 km
存在メッシュ：50325662231値47.98777611049852距離0.14468426940260257 km
存在メッシュ：50325662232値65.20888555366187距離0.2893685387995886 km
存在メッシュ：50325662213値52.805145804160134距離0.1851577137438622 km
存在メッシュ：50325662142値30.766666666666666距離0.0 km
存在メッシュ：50325662232値65.20888555366187距離0.2893685387995886 km
存在メッシュ：50325662231値47.98777611049852距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値46.470569042602634距離0.14468426940260257 km
存在メッシュ：50325662142値

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.1092461349149距離0.14468426940260257 km
存在メッシュ：50325662142値23.45距離0.0 km
存在メッシュ：50325662213値49.88838751653368距離0.1851577137438622 km
存在メッシュ：50325662231値44.1092461349149距離0.14468426940260257 km
存在メッシュ：50325662142値23.45距離0.0 km
存在メッシュ：50325662232値64.76849226902782距離0.2893685387995886 km
存在メッシュ：50325662124値39.947839636325256距離0.11554041512042086 km
存在メッシュ：50325662213値49.88838751653368距離0.1851577137438622 km
存在メッシュ：50325662231値44.1092461349149距離0.14468426940260257 km
存在メッシュ：50325662232値64.76849226902782距離0.2893685387995886 km
存在メッシュ：50325662213値49.88838751653368距離0.1851577137438622 km
存在メッシュ：50325662142値23.45距離0.0 km
存在メッシュ：50325662232値64.76849226902782距離0.2893685387995886 km
存在メッシュ：50325662231値44.1092461349149距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.65719995844624距離0.14468426940260257 km
存在メッシュ：50325662142値25.066666666666666距離0.0 km
50325662213 is kddi 

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値49.86764433009227距離0.14468426940260257 km
存在メッシュ：50325662142値32.56666666666667距離0.0 km
存在メッシュ：50325662213値54.70735606517766距離0.1851577137438622 km
存在メッシュ：50325662231値49.86764433009227距離0.14468426940260257 km
存在メッシュ：50325662142値32.56666666666667距離0.0 km
存在メッシュ：50325662232値67.16862199284625距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値54.70735606517766距離0.1851577137438622 km
存在メッシュ：50325662231値49.86764433009227距離0.14468426940260257 km
存在メッシュ：50325662232値67.16862199284625距離0.2893685387995886 km
存在メッシュ：50325662213値54.70735606517766距離0.1851577137438622 km
存在メッシュ：50325662142値32.56666666666667距離0.0 km
存在メッシュ：50325662232値67.16862199284625距離0.2893685387995886 km
存在メッシュ：50325662231値49.86764433009227距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.266208025227634距離0.14468426940260257 km
存在メッシュ：50325662142値30.383333333333333距離0.0 km
存在メッシュ：50325662

...

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値34.19226992391545距離0.14468426940260257 km
存在メッシュ：50325662142値19.983333333333334距離0.0 km
存在メッシュ：50325662213値38.16702548388007距離0.1851577137438622 km
存在メッシュ：50325662231値34.19226992391545距離0.14468426940260257 km
存在メッシュ：50325662142値19.983333333333334距離0.0 km
存在メッシュ：50325662232値48.401206513945986距離0.2893685387995886 km
存在メッシュ：50325662124値31.330153819583543距離0.11554041512042086 km
存在メッシュ：50325662213値38.16702548388007距離0.1851577137438622 km
存在メッシュ：50325662231値34.19226992391545距離0.14468426940260257 km
存在メッシュ：50325662232値48.401206513945986距離0.2893685387995886 km
存在メッシュ：50325662213値38.16702548388007距離0.1851577137438622 km
存在メッシュ：50325662142値19.983333333333334距離0.0 km
存在メッシュ：50325662232値48.401206513945986距離0.2893685387995886 km
存在メッシュ：50325662231値34.19226992391545距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値42.7328746918943距離0.14468426940260257 km
存在メッシュ：50325662142値24

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値49.10520281240484距離0.14468426940260257 km
存在メッシュ：50325662142値32.63333333333333距離0.0 km
存在メッシュ：50325662213値53.71298287937667距離0.1851577137438622 km
存在メッシュ：50325662231値49.10520281240484距離0.14468426940260257 km
存在メッシュ：50325662142値32.63333333333333距離0.0 km
存在メッシュ：50325662232値65.57707229083692距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値53.71298287937667距離0.1851577137438622 km
存在メッシュ：50325662231値49.10520281240484距離0.14468426940260257 km
存在メッシュ：50325662232値65.57707229083692距離0.2893685387995886 km
存在メッシュ：50325662213値53.71298287937667距離0.1851577137438622 km
存在メッシュ：50325662142値32.63333333333333距離0.0 km
存在メッシュ：50325662232値65.57707229083692距離0.2893685387995886 km
存在メッシュ：50325662231値49.10520281240484距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.333854361144375距離0.14468426940260257 km
存在メッシュ：50325662142値30.333333333333332距離0.0 km
存在メッシュ：50325662

存在メッシュ：50325662231値14.443564215676624距離0.14468426940260257 km
存在メッシュ：50325662142値9.1距離0.0 km
存在メッシュ：50325662213値15.938353177601218距離0.1851577137438622 km
存在メッシュ：50325662231値14.443564215676624距離0.14468426940260257 km
存在メッシュ：50325662142値9.1距離0.0 km
存在メッシュ：50325662232値19.787128431145817距離0.2893685387995886 km
存在メッシュ：50325662124値13.367206312414758距離0.11554041512042086 km
存在メッシュ：50325662213値15.938353177601218距離0.1851577137438622 km
存在メッシュ：50325662231値14.443564215676624距離0.14468426940260257 km
存在メッシュ：50325662232値19.787128431145817距離0.2893685387995886 km
存在メッシュ：50325662213値15.938353177601218距離0.1851577137438622 km
存在メッシュ：50325662142値9.1距離0.0 km
存在メッシュ：50325662232値19.787128431145817距離0.2893685387995886 km
存在メッシュ：50325662231値14.443564215676624距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値19.08110840274934距離0.14468426940260257 km
存在メッシュ：50325662142値11.383333333333333距離0.0 km
存在メッシュ：50325662213値21.23445563330833距離0.1851577137438622 km
存在メッシュ：5

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.46561552121731距離0.14468426940260257 km
存在メッシュ：50325662142値27.666666666666668距離0.0 km
存在メッシュ：50325662213値49.1648915673532距離0.1851577137438622 km
存在メッシュ：50325662231値44.46561552121731距離0.14468426940260257 km
存在メッシュ：50325662142値27.666666666666668距離0.0 km
存在メッシュ：50325662232値61.26456437511584距離0.2893685387995886 km
50325662124 is kddi none!
存在メッシュ：50325662213値49.1648915673532距離0.1851577137438622 km
存在メッシュ：50325662231値44.46561552121731距離0.14468426940260257 km
存在メッシュ：50325662232値61.26456437511584距離0.2893685387995886 km
存在メッシュ：50325662213値49.1648915673532距離0.1851577137438622 km
存在メッシュ：50325662142値27.666666666666668距離0.0 km
存在メッシュ：50325662232値61.26456437511584距離0.2893685387995886 km
存在メッシュ：50325662231値44.46561552121731距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.04357343273816距離0.14468426940260257 km
存在メッシュ：50325662142値27.45距離0.0 km
存在メッシュ：50325662213値48.6853985

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値46.92795590569638距離0.14468426940260257 km
存在メッシュ：50325662142値28.95距離0.0 km
存在メッシュ：50325662213値51.9570430395167距離0.1851577137438622 km
存在メッシュ：50325662231値46.92795590569638距離0.14468426940260257 km
存在メッシュ：50325662142値28.95距離0.0 km
存在メッシュ：50325662232値64.90591181069486距離0.2893685387995886 km
存在メッシュ：50325662124値43.30664358632354距離0.11554041512042086 km
存在メッシュ：50325662213値51.9570430395167距離0.1851577137438622 km
存在メッシュ：50325662231値46.92795590569638距離0.14468426940260257 km
存在メッシュ：50325662232値64.90591181069486距離0.2893685387995886 km
存在メッシュ：50325662213値51.9570430395167距離0.1851577137438622 km
存在メッシュ：50325662142値28.95距離0.0 km
存在メッシュ：50325662232値64.90591181069486距離0.2893685387995886 km
存在メッシュ：50325662231値46.92795590569638距離0.14468426940260257 km


In [74]:
connection.close()

In [27]:
#CreateGeojsonFile